In [15]:
import pandas as pd

In [16]:
# Number of sheets in row / columns
ATLAS_DIMS = (6, 9)

# Approximate sheets margins, in % of the image width/height
#          LEFT   RIGHT  TOP    BOTTOM
#MARGINS = [0.064, 0.07, 0.055, 0.06]

MARGINS = [0.044, 0.044, 0.07, 0.07]

def _jacoubet_sheet_dims(band: int, axis) -> int:
    """Return the number of grid lines (vertical or horizontal) in a sheet of the Jacoubet's Atlas."""
    match axis:
        case 0:  # Rows
            return 6 - (1 if band == 0 else 0)
        case 1:  # Columns
            return 5
        case _:
            raise ValueError(f"Invalid axis {axis}")

In [17]:
import pandas as pd


def grid_offset_for_band(band: int, axis: int, sheet_dims) -> int:
    """Get the offset to apply for a given band to get the starting position of a sheet in that band."""
    match axis:
        case 0:  # Rows
            offset = band * sheet_dims(band, 0) - band - 1
        case 1:  # Columns
            offset = band * sheet_dims(band, 1) - band
        case _:
            raise ValueError("Invalid axis %", axis)
    return max(0, offset)


def get_sheet_mask(
    grid: pd.DataFrame, atlas_x: int, atlas_y: int, sheet_dims
) -> pd.Series:
    """Calculate the grid-level boolean mask for the sheet at (atlas_x, atlas_y)."""
    row_offset = grid_offset_for_band(atlas_y, 0, sheet_dims)
    column_offset = grid_offset_for_band(atlas_x, 1, sheet_dims)

    return (
        (grid.column >= column_offset)
        & (grid.row >= row_offset)
        & (grid.column < column_offset + sheet_dims(atlas_x, 1))
        & (grid.row < row_offset + sheet_dims(atlas_y, 0))
    )


def compute_sheets_masks(grid: pd.DataFrame, sheet_dims_f):
    # Retrieve the coordinates of the grid axes to construct
    # the indices for the rows and columns of the grid.
    grid_row_steps = grid.Y.unique().tolist()
    grid_col_steps = grid.X.unique().tolist()

    # Assign the appropriate row and column to each point on the grid
    grid["row"] = grid.Y.apply(lambda y: grid_row_steps.index(y))
    grid["column"] = grid.X.apply(lambda x: grid_col_steps.index(x))

    sheet_counter = 0

    # Generate and apply the masks for the sheets.
    # ATLAS_DIMS defines an "atlas-level" grid whose 2D points are map sheets.
    for y_atlas in range(ATLAS_DIMS[0]):
        for x_atlas in range(ATLAS_DIMS[1]):
            sheet_counter += 1
            mask = get_sheet_mask(grid, x_atlas, y_atlas, sheet_dims_f)
            grid[sheet_counter] = mask

In [18]:
def make_gcps(
    grid: pd.DataFrame,
    sheet_ix: int,
    sheet: pd.Series,
    margins: list[int],
    sheet_dims_func,
):
    # mapX,mapY,sourceX,sourceY,enable,dX,dY,residual
    mask = grid[sheet_ix + 1]
    gcps = (
        grid[mask][["X", "Y"]]
        .rename(columns={"X": "mapX", "Y": "mapY"})
        .reset_index(drop=True)
    )

    sourceX = []
    sourceY = []

    atlas_y = sheet_ix // ATLAS_DIMS[1]  # row
    atlas_x = sheet_ix % ATLAS_DIMS[1]  # col

    cells_per_row = sheet_dims_func(atlas_y, 0)
    cells_per_column = sheet_dims_func(atlas_y, 1)

    delta = (
        (sheet.width - margins[0] - margins[1]) / (cells_per_column - 1),
        (sheet.height - margins[2] - margins[3]) / (cells_per_row - 1),
    )
    for pix in gcps.index:
        col = pix // sheet_dims_func(atlas_y, 0)
        row = pix % sheet_dims_func(atlas_y, 0)
        sourceX.append(col * delta[0])
        sourceY.append(row * delta[1])
    print(sheet)
    print(margins)
    print(delta)
    print(sheet_dims_func(atlas_x, 0))
    print(sheet_dims_func(atlas_y, 1))
    gcps["sourceX"] = sourceX
    gcps["sourceY"] = sourceY
    gcps["sourceX"] = gcps.sourceX + margins[0]
    gcps["sourceY"] = gcps.sourceY * -1.0 - margins[2]
    gcps["enabled"] = 1
    return gcps

In [19]:
def main():
    # Load data
    grid = pd.read_csv("grid_jacoubet.csv")
    # Add sheet masks
    compute_sheets_masks(grid, sheet_dims_f=_jacoubet_sheet_dims)
    # Save results
    grid.to_csv("output/grid.csv", index=False)

    sheet_dims_f = _jacoubet_sheet_dims
    sheets = pd.read_csv("dimensions.csv", sep=";")
    for ix, sheet in sheets.iterrows():
        margins = [
            MARGINS[0] * sheet.width,
            MARGINS[1] * sheet.width,
            MARGINS[2] * sheet.height,
            MARGINS[3] * sheet.height,
        ]
        print(margins)
        gcps = make_gcps(grid, ix, sheet, margins, sheet_dims_f)
        gcps.to_csv(f"output/{sheet.sheet}.points", index=False)

In [20]:
main()

[532.312, 532.312, 550.62, 550.62]
sheet     Atlas_de_Jacoubet_-_01_et_02._Feuille_de_titre...
width                                                 12098
height                                                 7866
Name: 0, dtype: object
[532.312, 532.312, 550.62, 550.62]
(2758.344, 1691.19)
5
5
[282.304, 282.304, 477.47, 477.47]
sheet     Atlas_de_Jacoubet_-_03._Partie_de_la_commune_d...
width                                                  6416
height                                                 6821
Name: 1, dtype: object
[282.304, 282.304, 477.47, 477.47]
(1462.848, 1466.5149999999999)
6
5
[281.424, 281.424, 475.72, 475.72]
sheet     Atlas_de_Jacoubet_-_04._Partie_de_la_commune_d...
width                                                  6396
height                                                 6796
Name: 2, dtype: object
[281.424, 281.424, 475.72, 475.72]
(1458.288, 1461.1399999999999)
6
5
[281.424, 281.424, 468.02000000000004, 468.02000000000004]
sheet     Atlas_de_Jacoubet_